# 抽取式文本摘要生成

## tf idf

In [4]:
docA="The cat sat on my face"
docB="The dog sat on my bed"

In [5]:
bowA=docA.split()
bowB=docB.split()

In [6]:
bowB

['The', 'dog', 'sat', 'on', 'my', 'bed']

In [7]:
bowA

['The', 'cat', 'sat', 'on', 'my', 'face']

In [8]:
wordSet=set(bowA).union(set(bowB))

In [9]:
wordSet

{'The', 'bed', 'cat', 'dog', 'face', 'my', 'on', 'sat'}

In [17]:
a=[i for i in range(8)]
a={0,1,2,3,4,5,6,7}

In [20]:
wordDictA=dict.fromkeys(wordSet,0)
wordDictB=dict.fromkeys(wordSet,0)

In [21]:
wordDictA

{'sat': 0, 'The': 0, 'my': 0, 'face': 0, 'dog': 0, 'cat': 0, 'on': 0, 'bed': 0}

In [22]:
wordDictB

{'sat': 0, 'The': 0, 'my': 0, 'face': 0, 'dog': 0, 'cat': 0, 'on': 0, 'bed': 0}

In [25]:
for word in bowA:
    wordDictA[word]+=1
for word in bowB:
    wordDictB[word]+=1

In [26]:
import pandas as pd
pd.DataFrame([wordDictA,wordDictB])

,sat,The,my,face,dog,cat,on,bed
0,1,1,1,1,0,1,1,0
1,1,1,1,0,1,0,1,1


In [27]:
def computeTF(wordDict,bow):
    tfDict={}
    bowCount=len(bow)
    for word,count in wordDict.items():
        tfDict[word]=count/float(bowCount)
    return tfDict

In [28]:
tfBowA=computeTF(wordDictA,bowA)
tfBowB=computeTF(wordDictB,bowB)

In [29]:
tfBowA

{'sat': 0.16666666666666666,
 'The': 0.16666666666666666,
 'my': 0.16666666666666666,
 'face': 0.16666666666666666,
 'dog': 0.0,
 'cat': 0.16666666666666666,
 'on': 0.16666666666666666,
 'bed': 0.0}

In [30]:
tfBowB

{'sat': 0.16666666666666666,
 'The': 0.16666666666666666,
 'my': 0.16666666666666666,
 'face': 0.0,
 'dog': 0.16666666666666666,
 'cat': 0.0,
 'on': 0.16666666666666666,
 'bed': 0.16666666666666666}

In [36]:
#计算逆向文件频率
def computeIDF(docList):
    import math
    idfDict={}
    N=len(docList)
    
    idfDict=dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word,val in doc.items():
            if val>0:
                idfDict[word]+=1
    for word,val in idfDict.items():
        idfDict[word]=math.log10(N/float(val))
        
    return idfDict

In [37]:
idfs=computeIDF([wordDictA,wordDictB])

In [38]:
idfs

{'sat': 0.0,
 'The': 0.0,
 'my': 0.0,
 'face': 0.3010299956639812,
 'dog': 0.3010299956639812,
 'cat': 0.3010299956639812,
 'on': 0.0,
 'bed': 0.3010299956639812}

In [39]:
def computeTFIDF(tfBow,ifds):
    tfidf={}
    for word,val in tfBow.items():
        tfidf[word]=val*idfs[word]
    return tfidf

In [40]:
tfidfBowB=computeTFIDF(tfBowB,idfs)

In [41]:
tfidfBowA

{'sat': 0.0,
 'The': 0.0,
 'my': 0.0,
 'face': 0.050171665943996864,
 'dog': 0.0,
 'cat': 0.050171665943996864,
 'on': 0.0,
 'bed': 0.0}

In [42]:
tfidfBowB

{'sat': 0.0,
 'The': 0.0,
 'my': 0.0,
 'face': 0.0,
 'dog': 0.050171665943996864,
 'cat': 0.0,
 'on': 0.0,
 'bed': 0.050171665943996864}

In [43]:
pd.DataFrame([tfidfBowA,tfidfBowB])

,sat,The,my,face,dog,cat,on,bed
0,0.0,0.0,0.0,0.050172,0.000000,0.050172,0.0,0.000000
1,0.0,0.0,0.0,0.000000,0.050172,0.000000,0.0,0.050172


In [51]:
#导入TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
#实例化tf实例（具体参数还挺多的挺复杂）
tv=TfidfVectorizer(use_idf=True,smooth_idf=True,norm=None)


In [53]:
#输入训练集举证，每行表示一个文本
train = ["Chinese Beijing Chinese",
        "Chinese Chinese Shanghai",
        "Chinese Macao",
        "Tokyo Japan Chinese"]

In [54]:
#训练，构建词汇表以及词项idf值，并将输入文本列表转换成VSM矩阵形式
tv_fit=tv.fit_transform(train)

In [55]:
#查看一下构建的词汇表
tv.get_feature_names()

['beijing', 'chinese', 'japan', 'macao', 'shanghai', 'tokyo']

In [56]:
#查看输入文本列表的VSM矩阵
tv_fit.toarray()

array([[1.91629073, 2.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 2.        , 0.        , 0.        , 1.91629073,
        0.        ],
       [0.        , 1.        , 0.        , 1.91629073, 0.        ,
        0.        ],
       [0.        , 1.        , 1.91629073, 0.        , 0.        ,
        1.91629073]])

## Sentence embeddings--SIF （拿预训练的词向量加权平均的一种方式）

In [58]:
import numpy as np
from sklearn.decomposition import PCA
from typing import List

In [59]:
class Word:
    def __init__(self,text,vector):
        self.text=text
        self.vector=vector
        
    def __str__(self):
        return self.text+':'+str(self.vector)
    

In [60]:
class Sentence:
    def __init__(self,word_list):
        self.word_list=word_list
    def len(self):
        return len(self.word_list)
    def __str__(self):
        word_str_list=[word.text for word in self.word_list]
        return ' '.join(word_str_list)
    def __repr__(self):
        return self.__str__()
    

In [61]:
def get_word_frequency(word_text):
    return 0.0001

In [64]:
def sentence_to_vec(sentence_list, embedding_size, a):
    sentence_set = []
    for sentence in sentence_list:
        vs = np.zeros(embedding_size)
        sentence_length = sentence.len()
        for word in sentence.word_list:
            a_value = a / (a + get_word_frequency(word.text))  # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word.vector)) # vs += sif * word_vector
        
        vs = np.divide(vs, sentence_length)  # weighted average
        sentence_set.append(vs)  # add to our existing re-calculated set of sentences
    
    # calculate PCA of this sentence set
    pca = PCA()
    pca.fit(np.array(sentence_set))
    u=pca.components_[0]#the PCA vector
    u=np.multiply(u,np.transpose(u)) #u*uT
    
    if len(u) < embedding_size:
        for i in range(embedding_size-len(u)):
            u=np.append(u,0) #add needed extension for multiplication below
    #resulting sentence vectors,vs=vs-u*uT*vs
    sentence_vecs=[]
    for vs in sentence_set:
        sub=np.multiply(u,vs)
        sentence_vecs.append(np.subtract(vs,sub))
    return sentence_vecs
                       

## textrank

In [77]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [78]:
df=pd.read_csv(r'D:\数据\数据集\网球类文章\tennis_articles_v4.csv',encoding='utf-8')

In [79]:
df

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Federer won the Swiss Indoors last week by bea...,https://www.express.co.uk/sport/tennis/1038186...


In [84]:
sentences=[]
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

In [81]:
sentences

[['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
  "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
  'I think everyone knows this is my job here.',
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
  "I'm a pretty competitive girl.",
  "I say my hellos, but I'm not sending any players flowers as well.",
  "Uhm, I'm not really friendly or close to many players.",
  "I have not a lot of friends away from the courts.'",
  'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
  "Is it different on the men's tour than the women's tour?",
  "'No, 

In [85]:
#把所有句子都放到了一个列表中
sentences=[y for x in sentences for y in x]

In [86]:
sentences

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at all.

In [87]:
#remove punctuations,numbers and special characters
clean_sentences=pd.Series(sentences).str.replace('[^a-zA-Z]',' ')

#make alphabets lowercase
chean_sentences=[s.lower() for s in clean_sentences]

In [88]:
nltk.download('stopwords')#one time execution

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [89]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

In [90]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [95]:
def remove_stopwords(sen):
    sen_new=' '.join([i for i in sen if i not in stop_words])
    return sen_new

In [96]:
clean_sentences=[remove_stopwords(r.split()) for r in clean_sentences]

In [99]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=fa3c428082e68cb7d5345442317fa8cd44b536c21fd88f03bdbc45e6d19e9a7d
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [100]:
#download pretrained Glove word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [102]:
!unzip glove*.zip

'unzip' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [104]:
#Extract word vectors
word_embeddings={}
f=open(r'D:\数据\数据集\glove词向量\glove.6B.50d.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    word_embeddings[word]=coefs
f.close()

In [103]:
clean_sentences

['Maria Sharapova basically friends tennis players WTA Tour',
 'The Russian player problems openly speaking recent interview said I really hide feelings much',
 'I think everyone knows job',
 'When I courts I court playing I competitor I want beat every single person whether locker room across net So I one strike conversation weather know next minutes I go try win tennis match',
 'I pretty competitive girl',
 'I say hellos I sending players flowers well',
 'Uhm I really friendly close many players',
 'I lot friends away courts',
 'When said really close lot players something strategic',
 'Is different men tour women tour',
 'No',
 'I think sport mean friends everyone categorized tennis player going get along tennis players',
 'I think every person different interests',
 'I friends completely different jobs interests I met different parts life',
 'I think everyone thinks tennis players greatest friends',
 'But ultimately tennis small part',
 'There many things interested',
 'BASEL Switz

In [105]:
sentence_vectors=[]
for i in clean_sentences:
    if len(i) !=0:
        v=sum([word_embeddings.get(w,np.zeros((50,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v=np.zeros((50,))
    sentence_vectors.append(v)

In [106]:
len(sentence_vectors)

119

In [108]:
#similarity matrix
sim_mat=np.zeros([len(sentences),len(sentences)])

In [109]:
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
cosine_similarity(sentence_vectors[1].reshape(1,50),sentence_vectors[2].reshape(1,50))

array([[0.89011891]])

In [111]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i !=j:
            sim_mat[i,j]=cosine_similarity(sentence_vectors[i].reshape(1,50),sentence_vectors[j].reshape(1,50))[0,0]

In [114]:
import networkx as nx
nx_graph =nx.from_numpy_array(sim_mat)
scores=nx.pagerank(nx_graph)

In [116]:
ranked_sentences=sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)

In [118]:
ranked_sentences

[(0.00920752178637705,
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."),
 (0.009204776384448126,
  'Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.'),
 (0.009153406152764871,
  'Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.'),
 (0.009119054496741582,
  '"We also had the impression that at this stage it might be better to play matches than to train.'),
 (0.009110260552167597, 'Because it was not always easy in the last weeks.

In [117]:
#specify number of sentences to form the summary
sn=3

#Generator summary
for i in range(sn):
    print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
